In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model('modelo.h5')

img_path = 'vidrio.jpg'
img = image.load_img(img_path, target_size=(32, 32))

img_array = image.img_to_array(img)

img_array = np.expand_dims(img_array, axis=0)

img_array /= 255.0

prediccion = model.predict(img_array)

print(prediccion)

categoriaM = np.argmax(prediccion)

print(f'El índice de la categoría predicha es: {categoriaM} con una probabilidad de {prediccion[0][categoria_mayor_probabilidad]:.2f}')

In [ ]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

# Cargar el modelo
model = load_model('modelo.h5')

# Inicializar la captura de video (0 para la cámara por defecto)
cap = cv2.VideoCapture(0)

while True:
    # Capturar frame por frame
    ret, frame = cap.read()

    # Verificar si el frame fue capturado correctamente
    if not ret:
        break

    # Preprocesar el frame
    img = cv2.resize(frame, (32, 32))  # Ajusta el tamaño según lo que espera tu modelo
    img_array = np.expand_dims(img, axis=0)  # Expandir las dimensiones
    img_array = img_array / 255.0  # Normalizar la imagen si es necesario

    # Hacer la predicción
    prediccion = model.predict(img_array)
    categoriaM = np.argmax(prediccion)

    # Mostrar el índice de la categoría en la imagen
    cv2.putText(frame, f'Categoria: {categoriaM}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Mostrar el frame con la predicción
    cv2.imshow('Prediccion', frame)

    # Romper el loop con la tecla 'q' o el num 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
